In [2]:
import sys; sys.path.append('..')
import numpy as np
import fitparse
import pandas as pd
import math
import plotly.express as px

from utils.paths import data_folder

In [44]:
def parse_fitfile(
  filepath: str,
  columns: list[str],
  verbose: bool = False,
) -> pd.DataFrame:
  """Parse a .fit file into a format the simulator can use.

  Notes
  -----
  The file must include at least timestamp, distance, and altitude.
  """
  ff = fitparse.FitFile(filepath)
  df = {col: [] for col in columns}

  generator = ff.get_messages("record", with_definitions=True)
  messages = list(generator)

  for m in messages:
    if type(m) == fitparse.records.DefinitionMessage:
      continue

    record_data = m.get_values()

    if verbose:
      print(record_data.keys())

    if any([col not in record_data for col in df]):
      ts = record_data["timestamp"]
      if verbose:
        print(f"Skipping incomplete record @ {ts}")
      continue

    for col in df:
      df[col].append(record_data[col])

  return pd.DataFrame(df)


def preprocess_columns(df):
  """Preprocess grade."""
  df["dy"] = df.altitude.diff(1)
  df["dx"] = df.distance.diff(1)
  df["dlat"] = df.position_lat.diff(1)
  df["dlon"] = df.position_long.diff(1)
  df["grade"] = (df.dy / df.dx).fillna(0)
  df["theta"] = df.grade.map(math.atan).fillna(0)
  df["v2"] = df.speed ** 2
  df["v2_diff"] = df.v2.diff(1).fillna(0)
  df["speed_diff"] = df.speed.diff(1).fillna(0)
  df["dt"] = df.timestamp.diff(1).fillna(0)
  df["P_prev_sec"] = df.power.shift(1).fillna(0)
  df.dt = df.dt.map(lambda x: x.total_seconds() if type(x) == pd.Timedelta else x)
  return df


def estimate_tailwind(
  df: pd.DataFrame,
  m: float = 82,
  rho: float = 1.225,
  L: float = 0.05,
  Crr: float = 0.006,
  CdA: float = 0.3
):
  G = 9.81 # m/s2
  v = df.speed.to_numpy()
  dx = df.dx.to_numpy()
  P_legs = df.power.to_numpy()
  theta = df.theta.to_numpy()
  dv2 = df.v2_diff.to_numpy()
  dy = df.dy.to_numpy()
  dt = df.dt.to_numpy()

  W_legs = P_legs * (1 - L) * dt
  W_roll = m*G*np.cos(theta) * Crr * dx
  dE = 0.5*m*dv2 + m*G*dy

  a = np.ones(len(v))
  b = -2*v
  c = v**2 - 2*(W_legs - W_roll - dE)/(rho*CdA*dx)

  roots = (-b - np.sqrt(b**2 - 4*a*c)) / (2*a)

  return roots


def estimate_parameters(df: pd.DataFrame, L: float = 0.05):
  G = 9.81 # m/s2
  m = 72 + 10 # kg
  rho = 1.225 # kg/m3

  dv = df.speed_diff.to_numpy()
  v0 = df.speed.to_numpy() - df.speed_diff.to_numpy()
  dv2 = df.v2_diff.to_numpy()
  dx = df.dx.to_numpy()
  dy = df.dy.to_numpy()
  dt = df.dt.to_numpy()
  P_legs = df.power.to_numpy()
  theta = df.theta.to_numpy()

  # Change in kinetic energy + change in potential energy from previous row to current row (t-1 -> t).
  c1 = dt * P_legs
  b = 0.5*m*dv2 + m*G*dy - c1*(1-L)
  c2 = -dx * m * G * np.cos(theta)
  c3 = -0.5 * rho * dx * (v0**2 + v0*dv + (1/3)*dv**2)
  A = np.column_stack((c2, c3))

  x, residuals, rank, s = np.linalg.lstsq(A, b)

  yhat = A @ x

  return x, residuals, rank, s, yhat, b

In [54]:
cols = [
  'timestamp',
  'distance',
  'altitude',
  'position_lat',
  'position_long',
  'power',
  'speed',
]
filepath = data_folder("fit/Boston_Tri.fit")
# filepath = data_folder("fit/Great_Brook_Farm.fit")
# filepath = data_folder("fit/IM_Santa_Cruz_70.3.fit")
df = parse_fitfile(filepath, cols, verbose=False)
df = preprocess_columns(df)
df.drop(labels=[0], inplace=True)

# L = 0.05
# x, residuals, rank, s, y_hat, b = estimate_parameters(df, L=L)
# x

tw = estimate_tailwind(df, m=82, rho=1.225, L=0.04, Crr=0.004, CdA=0.4)
df["tailwind"] = tw
df["tailwind_sign"] = df.tailwind.map(lambda x: 1 if x > 0 else -1)
df = df[df.tailwind.abs() < 10]

fig = px.line(df, x="timestamp", y="tailwind")
fig.show()

/var/folders/0z/fnhwmrr91f362mk6_bndszhw0000gn/T/ipykernel_40985/1548360452.py:81: RuntimeWarning:

invalid value encountered in sqrt



In [6]:
df["yaw"] = 

,timestamp,distance,altitude,position_lat,position_long,power,speed,dy,dx,grade,theta,v2,v2_diff,speed_diff,dt,P_prev_sec
1,2023-09-10 14:40:11,252.67,0.8,440996882,-1455818800,288,7.522,0.0,6.81,0.000000,0.000000,56.580484,0.779584,0.052,1.0,289.0
2,2023-09-10 14:40:12,260.71,0.8,440996252,-1455818037,262,7.634,0.0,8.04,0.000000,0.000000,58.277956,1.697472,0.112,1.0,288.0
3,2023-09-10 14:40:13,268.72,0.8,440995653,-1455817300,241,7.883,0.0,8.01,0.000000,0.000000,62.141689,3.863733,0.249,1.0,262.0
4,2023-09-10 14:40:14,276.67,0.8,440994981,-1455816577,237,7.915,0.0,7.95,0.000000,0.000000,62.647225,0.505536,0.032,1.0,241.0
5,2023-09-10 14:40:15,285.02,0.8,440994216,-1455816018,279,8.338,0.0,8.35,0.000000,0.000000,69.522244,6.875019,0.423,1.0,237.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8774,2023-09-10 17:06:34,90061.26,-3.6,441005213,-1455828395,0,7.905,-0.4,8.21,-0.048721,-0.048683,62.489025,3.199025,0.205,1.0,0.0
8775,2023-09-10 17:06:35,90069.47,-4.2,441005802,-1455829196,0,8.135,-0.6,8.21,-0.073082,-0.072952,66.178225,3.689200,0.230,1.0,0.0
8776,2023-09-10 17:06:36,90078.16,-4.8,441006316,-1455830192,0,8.824,-0.6,8.69,-0.069045,-0.068935,77.862976,11.684751,0.689,1.0,0.0
8777,2023-09-10 17:06:37,90086.69,-5.2,441006818,-1455831112,0,8.047,-0.4,8.53,-0.046893,-0.046859,64.754209,-13.108767,-0.777,1.0,0.0
